we're using the navier stokes equation here however without explicitly simulating pressure. turns out simulating pressure is especially difficult due to... reasons? i think we dont have an easy equation to calculate pressure like we do for other things and it's easier to calculate what pressure should be rather than what it is which is interesting.

so the general idea is using the navier stokes equation but instead of calculating pressure, solve the navier stokes equation ignoring pressure. the result of that equation tells us what our wind movement in x,y looks like. we can then use that info to solve the poisson equation. from my understanding the poisson equations finds us pressure values that would make sense in the context of weather. so for example if theres a lot of wind moving to the right it will assign the right a high pressure value and the surrounding areas lower pressure.

with these "solved for" pressure values, we actually update the x,y wind movement with the pressure values to get a realistic? simulation of wind

In [9]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
timestep = 0.1
cellSize = 1
gridLength = 32

viscosity = 0.001
density = 1.2
pressureAlpha = 0.8

In [3]:
def resetWind():
    # staggered grid
    xWindMovement = np.random.uniform(-1, 1, (gridLength, gridLength+1))
    yWindMovement = np.random.uniform(-1, 1, (gridLength+1, gridLength))

    return xWindMovement, yWindMovement

xWindMovement, yWindMovement = resetWind()

# xWindMovement = np.arange(1, 26).reshape(5, 5)
# yWindMovement = np.arange(1, 26).reshape(5, 5)

# xWindMovement = np.random.uniform(-0.1, 0.1, (5, 5))
# yWindMovement = np.random.uniform(-0.1, 0.1, (5, 5))

# x = np.linspace(-1, 1, 5)
# y = np.linspace(-1, 1, 5)
# X, Y = np.meshgrid(x, y)

# xWindMovement = -Y  # Creates a counterclockwise vortex
# yWindMovement = X

In [60]:
def calculateNeighbors(data):
    # easternNeighbor = np.hstack((data[:, 1:], np.zeros((5,1))))
    # westernNeighbor = np.hstack((np.zeros((5,1)), data[:, :-1]))
    # northernNeighbor = np.vstack((np.zeros(5), data[:-1]))
    # southernNeighbor = np.vstack((data[1:], np.zeros(5)))

    # easternNeighbor = np.hstack((data[:, 1:], -data[:, -1].reshape(5,1)))
    # westernNeighbor = np.hstack((-data[:, -1].reshape(5,1), data[:, :-1]))
    # northernNeighbor = np.vstack((-data[-1].reshape(1,5), data[:-1]))
    # southernNeighbor = np.vstack((data[1:], -data[1].reshape(1,5)))

    # wrap around
    easternNeighbor = np.hstack((data[:, 1:], data[:, :1]))
    westernNeighbor = np.hstack((data[:, -1:], data[:, :-1]))
    northernNeighbor = np.vstack((data[-1:], data[:-1]))
    southernNeighbor = np.vstack((data[1:], data[:1]))

    # use boundary velocities
    # easternNeighbor = np.hstack((data[:, 1:], data[:, -1].reshape(-1,1)))
    # westernNeighbor = np.hstack((data[:, 0].reshape(-1,1), data[:, :-1]))
    # northernNeighbor = np.vstack((data[0].reshape(1,-1), data[:-1]))
    # southernNeighbor = np.vstack((data[1:], data[-1].reshape(1,-1)))

    return (northernNeighbor, easternNeighbor, southernNeighbor, westernNeighbor)

def divergence(xVelocity, yVelocity):
    n,e,s,w = calculateVelocityNeighbor(xVelocity, yVelocity)
    return (e-w)/(cellSize*2)+ (n-s)/(cellSize*2)

def calculateVelocityNeighbor(xWind, yWind):
    north = np.diff(yWind, axis=0)
    east = np.diff(xWind, axis=1)
    south = -north
    west = -east
    return north, east, south, west

def fixVelocityShapeForOtherVelocity(wind, forVelocityStr):
    if forVelocityStr == 'x':
        prependedWind = np.hstack((wind[:, -1].reshape(-1, 1), wind))
        return np.diff(prependedWind, axis=0)
    else:
        prependedWind = np.vstack((wind[-1], wind))
        return np.diff(prependedWind, axis=1)

def averageWindAtCellCenter(xWind, yWind):
    n, e, _, _ = calculateVelocityNeighbor(xWind, yWind)
    return e/2, n/2

In [36]:
print(xWindMovement)

[[-0.67975772  0.6045068  -0.43425199 ... -0.65212739 -0.56267904
   0.39642312]
 [-0.40330893 -0.07958268 -0.87162528 ... -0.78308373  0.32380171
  -0.32208195]
 [-0.36523779 -0.65172061 -0.0791363  ... -0.81431042  0.34585534
   0.9647486 ]
 ...
 [-0.14846383  0.37748732  0.11906283 ... -0.98079766  0.38294354
   0.62003457]
 [ 0.67732468 -0.80522109 -0.0622079  ...  0.66772967  0.11655621
  -0.20591081]
 [ 0.87747898 -0.96875792 -0.14776538 ...  0.74851385  0.88898634
   0.44947535]]


In [59]:
np.vstack((xWindMovement[-1], xWindMovement))

array([[ 0.87747898,  0.87747898, -0.96875792, ...,  0.74851385,
         0.88898634,  0.44947535],
       [-0.67975772, -0.67975772,  0.6045068 , ..., -0.65212739,
        -0.56267904,  0.39642312],
       [-0.40330893, -0.40330893, -0.07958268, ..., -0.78308373,
         0.32380171, -0.32208195],
       ...,
       [-0.14846383, -0.14846383,  0.37748732, ..., -0.98079766,
         0.38294354,  0.62003457],
       [ 0.67732468,  0.67732468, -0.80522109, ...,  0.66772967,
         0.11655621, -0.20591081],
       [ 0.87747898,  0.87747898, -0.96875792, ...,  0.74851385,
         0.88898634,  0.44947535]])

In [52]:
xWindMovement[:, 0].reshape(-1, 1)

array([[-0.67975772],
       [-0.40330893],
       [-0.36523779],
       [-0.12646174],
       [-0.93607035],
       [-0.76129427],
       [ 0.45813581],
       [ 0.95675448],
       [ 0.23746025],
       [ 0.6387387 ],
       [ 0.64350554],
       [-0.30196057],
       [ 0.27314054],
       [ 0.25802646],
       [ 0.39765687],
       [-0.06231303],
       [ 0.5605871 ],
       [-0.07516681],
       [-0.71431486],
       [-0.27658492],
       [-0.66319327],
       [ 0.71703873],
       [-0.88606202],
       [ 0.24523439],
       [-0.95863464],
       [ 0.09954928],
       [ 0.09575364],
       [-0.2138557 ],
       [ 0.33425353],
       [-0.14846383],
       [ 0.67732468],
       [ 0.87747898]])

In [61]:
fixVelocityShapeForOtherVelocity(yWindMovement, 'x').shape

(32, 33)

In [10]:
print(xWindMovement.shape)
print(calculateNeighbors(yWindMovement)[1].shape)

(32, 33)
(33, 32)


In [ ]:
np.add(np.array([[1, 2], [3, 4]]), 0)

array([[1, 2],
       [3, 4]])

In [5]:
xWind = np.zeros((32, 32))
yWind = np.zeros((32, 32))

# Inject static divergence blob
xWind[16, 17] += 0.5
xWind[16, 15] -= 0.5
yWind[17, 16] -= 0.5
yWind[15, 16] += 0.5

# Plot initial divergence map
div = divergence(xWind, yWind)
div[16, 14:19]

ValueError: operands could not be broadcast together with shapes (32,31) (31,32) 

In [ ]:
# n,e,s,w = calculateNeighbors(xWindMovement)
# xAdvection = -1 * xWindMovement * (e - w)/(2*cellSize) - yWindMovement * (n - s)/(2*cellSize)
# xDiffusion = viscosity * ((e - 2*xWindMovement + w)/(cellSize**2) + (n - 2*xWindMovement + s)/(cellSize**2))
# xPredictedWind = xWindMovement + timestep * (xAdvection + xDiffusion)

# n,e,s,w = calculateNeighbors(yWindMovement)
# yAdvection = -1 * xWindMovement * (e - w)/(2*cellSize) - yWindMovement * (n - s)/(2*cellSize)
# yDiffusion = viscosity * ((e - 2*yWindMovement + w)/(cellSize**2) + (n - 2*yWindMovement + s)/(cellSize**2))
# yPredictedWind = yWindMovement + timestep * (yAdvection + yDiffusion)

In [ ]:
# prevPressure = np.full((5,5), np.inf)
# pressure = np.zeros((5, 5))
# stabalizedThresold = 0.001
# numIters = 0
# while np.abs(pressure - prevPressure).max() > stabalizedThresold:
#     summedNeighborPressure = np.add.reduce(np.array(calculateNeighbors(pressure)))
#     cellDivergence = divergence(xPredictedWind, yPredictedWind)
#     currentPressure = (summedNeighborPressure - cellDivergence)/4

#     prevPressure = pressure
#     pressure = currentPressure
#     numIters += 1

# print(f'After {numIters} iterations...')
# print(pressure)

After 79 iterations...
[[ -51.81513362  -59.30903544  -46.88127949  -15.02611314   78.51319075]
 [-163.952224   -190.54109838 -171.19177932 -125.73785353   83.07819115]
 [-337.45371372 -423.7138497  -407.60950528 -373.81456776  -20.46370233]
 [-532.15035116 -729.2535408  -741.72136429 -781.45018929 -341.12000267]
 [-135.8947598  -465.43041302 -484.57432118 -633.14731738 -512.56664877]]


In [ ]:
# n,e,s,w = calculateNeighbors(pressure)

# xPressureGradient = (e-w)/(2*cellSize)
# yPressureGradient = (n-s)/(2*cellSize)

# # we multiply by timestep to scale the pressure according to timestep - more timesteps passing = greater movement as a result
# # we divide by density to scale the pressure according to how difficult it is to move air. for a very dense substance, it takes greater force, greater pressure to move it compared to low density fluid which will move easily
# xWindMovement = xPredictedWind - timestep/density * xPressureGradient
# yWindMovement = yPredictedWind - timestep/density * yPressureGradient

In [6]:
# def plotData(data1, data2, title1, title2, vmin=None, vmax=None):
#     fig, axes = plt.subplots(1, 2, figsize=(10, 5))  # Create 1 row, 2 columns

#     # Determine color scale limits if not provided
#     if vmin is None or vmax is None:
#         vmin = min(np.min(data1), np.min(data2))
#         vmax = max(np.max(data1), np.max(data2))

#     # Plot with consistent color scale
#     im1 = axes[0].imshow(data1, cmap='viridis', vmin=vmin, vmax=vmax)
#     axes[0].set_title(title1)
#     fig.colorbar(im1, ax=axes[0])

#     im2 = axes[1].imshow(data2, cmap='viridis', vmin=vmin, vmax=vmax)
#     axes[1].set_title(title2)
#     fig.colorbar(im2, ax=axes[1])

#     plt.show()

def plotData(xWind, yWind, pressure, title='title', vmin=-1, vmax=1):
    # fig, axes = plt.subplots(1, 1, figsize=(10, 10))
    fig, axes = plt.subplots(1, 4, figsize=(15, 5))

    # X-component heatmap
    im1 = axes[0].imshow(xWind, cmap='RdBu', vmin=vmin, vmax=vmax, origin='upper')
    axes[0].set_title('X Wind')
    fig.colorbar(im1, ax=axes[0])

    # Y-component heatmap
    im2 = axes[1].imshow(yWind, cmap='RdBu', vmin=vmin, vmax=vmax, origin='upper')
    axes[1].set_title('Y Wind')
    fig.colorbar(im2, ax=axes[1])

    # Pressure heatmap
    im3 = axes[2].imshow(pressure, cmap='RdBu', vmin=vmin, vmax=vmax, origin='upper')
    axes[2].set_title('Pressure')
    fig.colorbar(im3, ax=axes[2])

    # Combined quiver plot
    avgX, avgY = averageWindAtCellCenter(xWind, yWind)
    ny, nx = avgX.shape
    X, Y = np.meshgrid(np.arange(nx), np.arange(ny))

    # axes[2].quiver(X, Y, xWind, yWind, angles='xy', scale_units='xy', scale=1, color='black')
    axes[3].streamplot(X, Y, avgX, -avgY, color='black')
    axes[3].invert_yaxis() # invert y axis since streamplot assumes last row is y=0
    axes[3].set_title(f'{title}\nWind Direction')
    # axes.streamplot(X, Y, xWind, yWind, color='black')
    # axes.set_title(f'{title}\nWind Direction')

    plt.tight_layout()
    plt.show()

In [77]:
def predictWind(xWind, yWind, calculateStr):
    if calculateStr == 'x':
        mainWind = xWind
        yWind = fixVelocityShapeForOtherVelocity(yWind, 'x')
    else:
        mainWind = yWind
        xWind = fixVelocityShapeForOtherVelocity(xWind, 'y')

    n, e, s, w = calculateNeighbors(mainWind)

    # Upwind Differencing (prevents unrealistic reversals)
    advection_x = np.where(xWind > 0,
                           -xWind * (mainWind - w) / cellSize, # wind coming from left
                           -xWind * (e - mainWind) / cellSize) # wind coming from right
    advection_y = np.where(yWind > 0,
                           -yWind * (mainWind - s) / cellSize,
                           -yWind * (n - mainWind) / cellSize)

    diffusion = viscosity * ((e - 2*mainWind + w)/(cellSize**2) + (n - 2*mainWind + s)/(cellSize**2))

    return mainWind + timestep * (advection_x + advection_y + diffusion)

'''
divergence of grid cell tells us how much air is moving IN to the cell vs OUT. for our sim, we want divergence to be 0 meaning that roughly equal amounts of air move in and out. this allows us to say density is the same everywhere
since no cell can have great divergence without high density in that cell

we want to find pressure values such that cells with higher divergence have higher pressure and cells with low divergence has low pressure. this means that cells with high divergence will have high pressure zones
that move to low pressure zones which are low divergence cells. it's a little bit more precise than that since the solve has to exactly find values such that all cells have divergence 0.
'''
# def solvePressure(xWind, yWind):
#     pressure = np.zeros((gridLength, gridLength))
#     max_iters = 500  # Increase iteration count
#     tolerance = 1e-8  # Stricter stopping condition

#     for i in range(max_iters):
#         prevPressure = pressure.copy()
#         cellDivergence = divergence(xWind, yWind)

#         # Gauss-Seidel Iteration (directly modifies pressure)
#         n, e, s, w = calculateNeighbors(pressure)
#         # pressure = 0.25 * (e + w + n + s - cellDivergence)
#         pressure = (1 - pressureAlpha) * pressure + pressureAlpha * (0.25 * (e + w + n + s - cellDivergence))

#         if np.linalg.norm(pressure - prevPressure) < tolerance:
#             break

#         if i % 10 == 0:
#             print(f"Iteration {i}: Max divergence = {np.max(np.abs(cellDivergence))}")


#     return pressure
def solvePressure(cellDivergence):
    pressure = np.zeros((gridLength, gridLength))
    max_iters = 500  # Increase iteration count
    tolerance = 1e-8  # Stricter stopping condition

    for i in range(max_iters):
        prevPressure = pressure.copy()

        # Gauss-Seidel Iteration (directly modifies pressure)
        n, e, s, w = calculateNeighbors(pressure)
        # pressure = 0.25 * (e + w + n + s - cellDivergence)
        pressure = (1 - pressureAlpha) * pressure + pressureAlpha * (0.25 * (e + w + n + s - cellDivergence))

        if np.linalg.norm(pressure - prevPressure) < tolerance:
            break

        # if i % 10 == 0:
        #     print(f"Iteration {i}: Max divergence = {np.max(np.abs(cellDivergence))}")
            # print(f"Iteration {i}: Max Pressure difference = {np.max(np.abs(pressure - prevPressure))}")

    return pressure


def updateWind(wind, pressure, windStr):
    if windStr == 'x':
        wrapAroundPressure = np.hstack((pressure[:, -1].reshape(-1, 1), pressure, pressure[:, 0].reshape(-1, 1)))
        pressureGradient = np.diff(wrapAroundPressure, axis=1)/cellSize
    else:
        wrapAroundPressure = np.vstack((pressure[-1], pressure, pressure[0]))
        pressureGradient = np.diff(wrapAroundPressure, axis=0)/cellSize

    return wind - (timestep/density) * pressureGradient

def getCFLTimestep(xWind, yWind):
    return cellSize / max(np.max(np.abs(xWind)), np.max(np.abs(yWind)))

In [ ]:
xWindMovement, yWindMovement = resetWind()
plotData(xWindMovement, yWindMovement, np.zeros((gridLength, gridLength)))

for i in range(100):
    timestep = getCFLTimestep(xWindMovement, yWindMovement)
    # print(f"Timestep: {timestep}")

    xPredictedWind = predictWind(xWindMovement, yWindMovement, 'x')
    yPredictedWind = predictWind(xWindMovement, yWindMovement, 'y')

    # print(f"After advection max u = {np.max(xPredictedWind)}, min u = {np.min(xPredictedWind)}")


    div = divergence(xPredictedWind, yPredictedWind)
    print(f"After advection: max divergence = {np.max(np.abs(div))}")

    pressure = solvePressure(divergence(xPredictedWind, yPredictedWind))

    # print(f"Max pressure: {np.max(pressure)}, Min pressure: {np.min(pressure)}")

    xWindMovement = updateWind(xPredictedWind, pressure, 'x')
    yWindMovement = updateWind(yPredictedWind, pressure, 'y')

    div = divergence(xWindMovement, yWindMovement)
    print(f"After projection: max divergence = {np.max(np.abs(div))}")


    # print(f"Max divergence: {np.max(np.abs(divergence(xWindMovement, yWindMovement)))}")
    # print(f"Max pressure: {np.max(pressure)}, Min pressure: {np.min(pressure)}")

    plotData(xWindMovement, yWindMovement, pressure)

In [17]:
t = calculateVelocityNeighbor(xWindMovement, yWindMovement)
print(t[0].shape)
print(t[1].shape)

(32, 32)
(32, 32)


In [ ]:
# Setup grid
xWind = np.zeros((32, 32+1))
yWind = np.zeros((32+1, 32))

In [ ]:
n,e,_,_ = calculateVelocityNeighbor(xWind, yWind)
ny, nx = n.shape
X, Y = np.meshgrid(np.arange(ny), np.arange(nx))
print(X.shape)
print(Y.shape)
print(n.shape)
print(e.shape)

(32, 32)
(32, 32)
(32, 32)
(32, 32)


In [55]:
pressure = [    1,   2,   3,   4 ]
velocity = np.array([[0.5, 1.5, 2.5, 3.5, 4.5],
                     [10, 12, 15, 20, 27]])

print(np.diff(velocity, axis=1))
print(np.diff(velocity)*-1)

[[1. 1. 1. 1.]
 [2. 3. 5. 7.]]
[[-1. -1. -1. -1.]
 [-2. -3. -5. -7.]]


In [ ]:
calculateVelocityNeighbor(velocity, velocity)

(array([[ 9.5, 10.5, 12.5, 16.5, 22.5]]),
 array([[1., 1., 1., 1.],
        [2., 3., 5., 7.]]),
 array([[ -9.5, -10.5, -12.5, -16.5, -22.5]]),
 array([[-1., -1., -1., -1.],
        [-2., -3., -5., -7.]]))